<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy

In [ ]:
%%writefile Lotus.py
import scrapy
from scrapy.crawler import CrawlerProcess

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def start_requests(self):
        urls = [
            'https://quotes.toscrape.com'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'author': quote.css('small.author::text').get(),
                'text': quote.css('span.text::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [ ]:
%%bash
python Lotus.py

In [ ]:
%%bash
scrapy shell "https://www.lotuss.com.my/en"
response.css("title::text").getall()